  # **Fine-Tuning Mistral-7B on MetaMathQA-40k and Deployment of Model**



This step installs necessary Python packages. Notably, it includes bitsandbytes, transformers (Hugging Face library), peft, accelerate, and other dependencies for various tasks like training, fine-tuning, and evaluation.

#**Goal:**


The primary objective of this project was to fine-tune the Mistral 7B language model on a custom dataset and deploy it using Gradio for real-time interaction. The fine-tuned model aimed to showcase improved performance on a specific task, while the deployment allowed users to interact with the model through a user-friendly interface.

In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q trl xformers wandb datasets einops gradio sentencepiece``

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging, TextStreamer
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os,torch, wandb, platform, gradio, warnings
from datasets import load_dataset
from trl import SFTTrainer
from huggingface_hub import notebook_login

In [ ]:
notebook_login()

## Load Dataset

In [ ]:
!pip install datasets
from datasets import load_dataset


MATH_dataset = load_dataset("IntellectusAI/IndianStatues")

Generating train split:   0%|          | 0/784 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/datasets/packaged_modules/csv/csv.py:195: DtypeWarning: Columns (327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550

In [ ]:
print(MATH_dataset)

DatasetDict({
    train: Dataset({
        features: ['prompt', 'result', 'section_descriptions', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25', 'Unnamed: 26', 'Unnamed: 27', 'Unnamed: 28', 'Unnamed: 29', 'Unnamed: 30', 'Unnamed: 31', 'Unnamed: 32', 'Unnamed: 33', 'Unnamed: 34', 'Unnamed: 35', 'Unnamed: 36', 'Unnamed: 37', 'Unnamed: 38', 'Unnamed: 39', 'Unnamed: 40', 'Unnamed: 41', 'Unnamed: 42', 'Unnamed: 43', 'Unnamed: 44', 'Unnamed: 45', 'Unnamed: 46', 'Unnamed: 47', 'Unnamed: 48', 'Unnamed: 49', 'Unnamed: 50', 'Unnamed: 51', 'Unnamed: 52', 'Unnamed: 53', 'Unnamed: 54', 'Unnamed: 55', 'Unnamed: 56', 'Unnamed: 57', 'Unnamed: 58', 'Unnamed: 59', 'Unnamed: 60', 'Unnamed: 61', 'Unnamed: 62', 'Unnamed:

## Data Splitting

In [ ]:
# Assuming you have a "train" split
train_dataset = MATH_dataset["train"]

# Specify the desired number of samples
desired_samples = 784

# Ensure that the desired number of samples is not greater than the total number of samples
desired_samples = min(desired_samples, len(train_dataset))

# Take the first desired_samples rows
selected_samples = train_dataset.select(list(range(desired_samples)))

# Specify the desired ratio for your train/test split (e.g., 80% train, 20% test)
train_ratio = 0.8

# Calculate the number of samples for the train split
num_samples_train = int(desired_samples * train_ratio)

# Create the train and test splits
train_split = selected_samples.select(list(range(num_samples_train)))
test_split = selected_samples.select(list(range(num_samples_train, desired_samples)))

# Now you have train_split and test_split with 10,000 rows


In [ ]:
print(train_split,test_split)

Dataset({
    features: ['prompt', 'result', 'section_descriptions', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25', 'Unnamed: 26', 'Unnamed: 27', 'Unnamed: 28', 'Unnamed: 29', 'Unnamed: 30', 'Unnamed: 31', 'Unnamed: 32', 'Unnamed: 33', 'Unnamed: 34', 'Unnamed: 35', 'Unnamed: 36', 'Unnamed: 37', 'Unnamed: 38', 'Unnamed: 39', 'Unnamed: 40', 'Unnamed: 41', 'Unnamed: 42', 'Unnamed: 43', 'Unnamed: 44', 'Unnamed: 45', 'Unnamed: 46', 'Unnamed: 47', 'Unnamed: 48', 'Unnamed: 49', 'Unnamed: 50', 'Unnamed: 51', 'Unnamed: 52', 'Unnamed: 53', 'Unnamed: 54', 'Unnamed: 55', 'Unnamed: 56', 'Unnamed: 57', 'Unnamed: 58', 'Unnamed: 59', 'Unnamed: 60', 'Unnamed: 61', 'Unnamed: 62', 'Unnamed: 63', 'Unnamed: 64', 'Unnamed

## Create a Prompt

In [ ]:
def create_prompt(sample):
    query = sample['prompt']
    response = sample['section_descriptions']
    prompt = f"<s>[INST] {query} [/INST]\n"

    # Include the response from the 'response' column
    prompt += f"{response}</s>"

    return prompt

prompt_example = create_prompt(train_split[0])
print(prompt_example)

<s>[INST] (a), Section 5 r/w 27 of the Arms Act. The gist of theprosecution case relevant for the purpose of this proceedingmay be stated thus: With the growth of industry, commerceand trade in and around the city of Mumbai which generatessubstantial quantity of wealth, there has been increase oforganised activities by gangs of anti-socials to extractmoney from affluent sections of society like developers,hoteliers and other businessmen by putting them in fear ofdeath and then to demand substantial sums of money commonlyknown as "Khadani" i.e. protection money. One such gangwas operating in the city under Amar Naik @ Bhai, who dieda couple of years before the decision in the case at anencounter with the police. The prosecution alleged that inpursuance of a criminal conspiracy between 15.1.1994 to16.5.1994 the accused persons and others of the gangembarked upon preparatory acts like procuring theinformation about the names of the builders of M/s KalpataruConstruction Company which was e

# **Load the base Mistral 7B model with quantization configurations.**


Quantization is a technique used to reduce the memory and computation requirements of a neural network model. It involves representing the model's weights and activations with fewer bits, typically lower-precision data types, such as 8-bit integers or even lower. This reduction in precision helps in compressing the model, making it more efficient for deployment on resource-constrained devices, including edge devices and mobile platforms. thats why we used Quantization.


The tokenizer corresponding to the Mistral 7B model is loaded to preprocess input text and prepare it for the model. Setting specific tokenizer parameters, such as padding_side, model_max_length, and enabling trust_remote_code, ensures consistency with the tokenization used during pre-training. Additionally, adjustments are made to include end-of-sequence (EOS) tokens by assigning eos_token to the pad_token and enabling the add_eos_token attribute. These configurations align the tokenizer with the model's expectations, facilitating accurate and consistent input processing during inference.

In [ ]:
# Load base model(Mistral 7B)
bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
)
model = AutoModelForCausalLM.from_pretrained(
   "mistralai/Mixtral-8x7B-Instruct-v0.1",
    quantization_config=bnb_config,
    device_map={"": 0}
)
model.config.use_cache = False
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    "mistralai/Mixtral-8x7B-Instruct-v0.1",
    padding_side="left",
     model_max_length=512,
    trust_remote_code=True)

tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.add_bos_token, tokenizer.add_eos_token


config.json:   0%|          | 0.00/720 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/92.7k [00:00<?, ?B/s]

model-00001-of-00019.safetensors:   0%|          | 0.00/4.89G [00:00<?, ?B/s]

model-00002-of-00019.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00019.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

(True, True)

In [ ]:
def generate_response(prompt):
    encoded_input = tokenizer.apply_chat_template(prompt, return_tensors="pt")
    # attention_mask = encoded_input['attention_mask']
    model_inputs = encoded_input.to('cuda')
    generated_ids = model.generate(model_inputs, max_new_tokens=10, do_sample=True)
    decoded = tokenizer.batch_decode(generated_ids)
    return decoded[0]


In [ ]:
messages = [
    {"role": "user", "content": '''[INST]Prosecution case in brief was that Tuliya Devi, daughter of informant Bhullu Rajbhar (PW-1) was married to Gullu 5 years ago.After marriage Tuliya Devi was treated with cruelty by her husband Gullu,
father-in-law Bal Chand and mother-in-law Ramwati for demand of dowry.
For this reason these three accused had committed murder of Tuliya Devi on 8.5.2010 by inflicting injuries on her body at their house situate in village Chaubepur, police station- Mardah, district Ghazipur
and tried to dispose of her dead body.After receiving knowledge of this incident, victim's father Bhullu (informant) had given a written report (Ex-Ka-1) to the police on the basis of which case crime
no. 689 of 2010 was registered.3.In the aforesaid case inquest of the dead body was performed on 08.05.2010 and postmortem of the deceased was conducted the same evening at 9:50 pm.
In the postmortem report several lacerated wounds and other marks of injuries were found on the dead body of the deceased.In postmortem (Ex-Ka-1) the doctor had opined that cause of
death was haemorrhage and shock as a result of the said ante-mortem injuries.In this report approximate time of death was reported about 1 day from the time of postmortem.
After completion of investigation police had submitted charge-sheet against three accused Gullu, Bal Chand and Ramwati for offences under section 498-A, 304-B IPC and under section 3/4 Dowry Prohibition Act.
Accused had pleaded not guilty and claimed to be tried.5.In support of charges prosecution side examined PW-1, Bhullu
(informant), PW-2 Barmati (mother of the deceased), PW-3 - Constable Sunil Kumar Singh (who prepared chik FIR and registerd case),
PW-4 - Dr. Krishna Kumar Verma (who performed the postmortem), PW-5 Vinay Kumar Rai, Naib Tehsildar (for inquest report) and PW-6 Chrinjeev Nath Sinha (Investigation Officer).
Hon'ble Pramod Kumar Srivastava,J.(Delivered by Hon'ble Pramod KumarSrivastava, J.)1.This appeal has been preferred against the conviction and punishment dated 28.2.2012 passed in State Vs.
Gullu and others relating to Case Crime No. 689 of 2010, police station Mardah, district Ghazipur by which three accused Gullu, Bal Chand and Smt .Ramwati were convicted for charge under section
302/34 IPC and were punished with imprisonment for life and fine of Rs. 5000/- (in default of payment three months simple imprisonment).4.During trial all the accused were charged for offences under
section 498-A, 304 B IPC and under section 3/4 Dowry Prohibition Act, alongwith alternative charge framed later on for offence under section 302 IPC.6.After closure of prosecution evidence statement of accused were recorded in
 which they denied the facts of charge as well as evidence adduced against them, without any specific averments.Defence side had examined DW-1 Madan, r/o village Chaubeypur.7.After receiving evidence from both the
sides and after affording opportunity of hearing as well as considering the argument of the parties, learned Addl.Sessions Judge passed judgment dated 28.2.2012 by which all the accused were acquitted from the charges under
section 498-A, 304-B IPC and 3/4 Dowry Prohibition Act, but were convicted for the charge under section 302-IPC.Thereafter the trial court had afforded an opportunity of hearing on the point of quantum of sentence to the accused
and passed orders of punishment as above.Aggrieved by this judgment dated 28.2.2012 all the three accused have preferred the present appeal.8.Sri Sudist and Sri Janardan Singh Yadav appeared for appellants; and State was
represented by Mrs. Usha Kiran, AGA.9.A perusal of evidence
adduced during trial indicates that during post-mortem following ante-mortem injuries were found on the dead body of the deceased Tuliya Devi that were caused or occurred approximately at the time mentioned in the charge, i.e.,
anytime in the night of 8/9-5-2010: "1.Lacerated wound left side chin 6 cm x 1 cm mussel deep chin.", "2.Lacerated wound from right shoulder to just below right elbow 35 cm x 2 cm x bone deep.",
3.Abrasion 5cm x 2 cm just laterac of left eye.", "4.Abrasion on top of right shoulder 8 cm x 4 cm.", "5.Abrasion 20cm x 10 cm right side chest.", "6.brasion 8 cm x 2 cm right iliac chest.",
7.Lacerated wound 5 cm x 3 cm at middle of front of right of leg underlying bond was fractured. 8.Lacerated wound 6 cm x 3 cm just below right knee.", "9.Lacerated wound 8 cm x 3 cm below front of left kneel under lying
bone was fractured.10.PW-4 the doctor reported that the cause of death of victim-deceased Tuliya Devi was shock and haemmerhage due to the above mentioned ante-mortem injuries.Although the defence had adduced one witness
to indicate that the cause of death of the deceased may be accidental falling from the roof, and DW-1 Madan was examined in this regard but these facts could not be substantiated in the light of available evidence and circumstances.",
11.A perusal of the impugned judgment reveals that this finding of learned Additional Sessions Judge is correct that deceased Tuliya Devi had died due to injuries found on her body in the house of the accused-appellants at about the
time mentioned in the charge, and the accused-appellants were responsible for causing such injuries to Tuliya Devi.In these circumstances, the trial court had rightly reached to the conclusion that due to the above mentioned
deliberate caused injuries Tuliya Devi died, and accused-appellants are responsible for inflicting those homicidal injuries.12.Learned Additional Sessions Judge had considered facts and circumstances including evidence adduced
and reached to the conclusion that though there is no conclusive evidence relating to dowry death and demand of dowry, but injuries found on body of the deceased were not accidental.Trial Court found that those injuries were
homicidal, for inflicting of which accused persons were responsible, because it were only they who were present in their house when such injuries had occurred on the body of the deceased Tuliya Devi.13.Learned counsel appearing
for the appellants fairly admitted the contents of facts relating to charge, namely victim Tuliya Devi having succumbed to the injuries found on her body in the house of appellants.[/INST]'''}
]
response = generate_response(messages)
print(response)

Token indices sequence length is longer than the specified maximum sequence length for this model (1742 > 512). Running this sequence through the model will result in indexing errors
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] [INST]Prosecution case in brief was that Tuliya Devi, daughter of informant Bhullu Rajbhar (PW-1) was married to Gullu 5 years ago.After marriage Tuliya Devi was treated with cruelty by her husband Gullu,
father-in-law Bal Chand and mother-in-law Ramwati for demand of dowry.
For this reason these three accused had committed murder of Tuliya Devi on 8.5.2010 by inflicting injuries on her body at their house situate in village Chaubepur, police station- Mardah, district Ghazipur
and tried to dispose of her dead body.After receiving knowledge of this incident, victim's father Bhullu (informant) had given a written report (Ex-Ka-1) to the police on the basis of which case crime
no. 689 of 2010 was registered.3.In the aforesaid case inquest of the dead body was performed on 08.05.2010 and postmortem of the deceased was conducted the same evening at 9:50 pm.
In the postmortem report several lacerated wounds and other marks of injuries were found on the dead body of the deceased.In

# Prepare for K-Bit Training



Adapters are additional neural network components that can be fine-tuned to capture task-specific information without extensively modifying the pre-trained model. The prepare_model_for_kbit_training function readies the model for knowledge distillation, while the subsequent lines instantiate a LoraConfig object, specifying parameters such as adapter dimensions, dropout rates, and the target modules where adapters will be applied.

The get_peft_model function then integrates these adapters into the Mistral 7B model, allowing for the extraction and utilization of task-specific knowledge during the fine-tuning process. This facilitates the model's adaptability to the specific requirements of the downstream task.

In [ ]:
#Adding the adapters in the layers
model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
        r=16,
        lora_alpha=16,
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj"]
    )
model = get_peft_model(model, peft_config)

# Training Configuration

 The code sets up training arguments, initializes a SFTTrainer for training the model, and saves the trained model. The model is then pushed to the Hugging Face Model Hub for easy sharing and retrieval.

The TrainingArguments encapsulates key parameters and configurations for the training process. In this instance, it specifies the output directory for storing results, sets the number of training epochs to 1, defines the batch size and gradient accumulation steps, schedules saving of checkpoints, logs training progress every 10 steps, and incorporates additional settings such as learning rate, weight decay, and gradient clipping. The use of mixed-precision training (fp16) for faster computations and integration with WandB for real-time monitoring and reporting adds further versatility to the training setup.

In [ ]:
training_arguments = TrainingArguments(
    output_dir= "./results",
    num_train_epochs= 1,
    per_device_train_batch_size= 2,
    gradient_accumulation_steps=4,
    save_steps= 1000,
    logging_steps= 10,
    learning_rate= 2e-4,
    weight_decay= 0.001,
    fp16= True,
    max_grad_norm= 0.3,
    max_steps= -1,
    warmup_ratio= 0.3,
    group_by_length= True,
    lr_scheduler_type= "constant",
    report_to="wandb",

)


The SFTTrainer is initialized with the specified parameters for training the model. It involves the Mistral 7B model, a maximum sequence length of 256 tokens, training and evaluation datasets, as well as the configuration for adapter-based knowledge integration (peft_config). Additionally, the trainer incorporates a formatting function (create_prompt) for generating input prompts, uses the defined tokenizer, and adheres to the training arguments set in training_arguments, which include key details such as batch size, gradient accumulation steps, and optimization settings. The optional usage of packing is employed to handle variable-length sequences efficiently during training.

In [ ]:
trainer = SFTTrainer(
    model=model,
    max_seq_length = 2048,
    train_dataset=train_split,
    eval_dataset=test_split,
    peft_config=peft_config,
    formatting_func=create_prompt,
    # callbacks=[early_stopping],
    # dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing= True)


Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


# Start the training process.

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,1.432200
20,1.338900
30,1.354500
40,1.281800
50,1.304200
60,1.259700
70,1.305600
80,1.387700


TrainOutput(global_step=83, training_loss=1.33364166121885, metrics={'train_runtime': 2334.0947, 'train_samples_per_second': 0.285, 'train_steps_per_second': 0.036, 'total_flos': 3.8010069633623654e+17, 'train_loss': 1.33364166121885, 'epoch': 0.996996996996997})

#**Save the fine-tuned model and push it to the Hugging Face Model Hub.**


In [ ]:
trainer.save_model("mistral_finetuned_ILSI")

In [ ]:
notebook_login()

In [ ]:
model.push_to_hub("IntellectusAI/mistral_finetune_ILSI_MIXTRAL_8X7b")

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/54.6M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/IntellectusAI/mistral_finetune_ILSI_MIXTRAL_8X7b/commit/ba7fcba216d203391266eb0635df67458d87dda6', commit_message='Upload model', commit_description='', oid='ba7fcba216d203391266eb0635df67458d87dda6', pr_url=None, pr_revision=None, pr_num=None)

# Model Evaluation

In [ ]:
!pip install huggingface_hub

from huggingface_hub import login

login()


## Load Pre-trained Model for Evaluation

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

base_model_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,  # Mistral, same as before
    quantization_config=bnb_config,  # Same quantization config as before
    device_map="auto",
    trust_remote_code=True,
    use_auth_token=True
)

eval_tokenizer = AutoTokenizer.from_pretrained(base_model_id, add_bos_token=True, trust_remote_code=True)

/opt/conda/lib/python3.10/site-packages/transformers/models/auto/auto_factory.py:468: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/19 [00:00<?, ?it/s]

## Generate Response with the Fine-tuned Model

In [ ]:
from peft import PeftModel

ft_model = PeftModel.from_pretrained(base_model, "IntellectusAI/mistral_finetune_ILSI_MIXTRAL_8X7b")

adapter_config.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/54.6M [00:00<?, ?B/s]

In [ ]:
# eval_prompt = "<s>[INST] The case was of a 29-year-old female in an arranged marriage with demands for additional dowry. She was beaten up regularly by her husband and FIL. Following the beatings, she began to have some neurological problems. When she became pregnant with a female child (sex determination was done illegally) her in-laws and husband pressurized her to abortion it, which she refused and was allegedly administered some unknown substance/poison by her MIL.During this time there were she mentioned 2 specific episodes of violence by her husband and FIL when she was hit on the neck with danda and belt following which her condition start deteriorating. She was taken to a local hospital who referred her to Neurology Department of AIIMS. During her treatment she was put on ventilator and kept in ICU for 2 months.She alleges that even while admitted, she was subjected to multiple episodes of violence by her husband and FIL who threatened to remove the ventilator if her father didn’t pay additional dowry.She was discharged after 6 months with following diagnosis: LMN Quadriparesis with diaphragmatic weakness and ventilator dependence. She was not taken care of properly in her in laws house, so she went back to her own home and currently she is bedridden on ventilator and residing at her parents’ house.She has lodged an FIR with the police alleging domestic violence and torture with demands for additional dowry.A medical board of doctors was formed to opine whether her condition was a result of physical violence meted out her. Her treatment papers were reviewed, and in view of unknown substance administered to the victim, samples were collected for toxicological analysis.Specify the provisions of Indian Penal Code as they apply to the above case.and mention section which is relevant to this case.[/INST]"
# model_input = eval_tokenizer(eval_prompt, return_tensors="pt").to("cuda")

# ft_model.eval()
# with torch.no_grad():
    print(eval_tokenizer.decode(ft_model.generate(**model_input, max_new_tokens=4096, repetition_penalty=1.15)[0], skip_special_tokens=True))


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST] The case was of a 29-year-old female in an arranged marriage with demands for additional dowry. She was beaten up regularly by her husband and FIL. Following the beatings, she began to have some neurological problems. When she became pregnant with a female child (sex determination was done illegally) her in-laws and husband pressurized her to abortion it, which she refused and was allegedly administered some unknown substance/poison by her MIL.During this time there were she mentioned 2 specific episodes of violence by her husband and FIL when she was hit on the neck with danda and belt following which her condition start deteriorating. She was taken to a local hospital who referred her to Neurology Department of AIIMS. During her treatment she was put on ventilator and kept in ICU for 2 months.She alleges that even while admitted, she was subjected to multiple episodes of violence by her husband and FIL who threatened to remove the ventilator if her father didn’t pay additional

In [ ]:
# eval_prompt = "<s>[INST] The case was of a 29-year-old female in an arranged marriage with demands for additional dowry. She was beaten up regularly by her husband and FIL. Following the beatings, she began to have some neurological problems. When she became pregnant with a female child (sex determination was done illegally) her in-laws and husband pressurized her to abortion it, which she refused and was allegedly administered some unknown substance/poison by her MIL.During this time there were she mentioned 2 specific episodes of violence by her husband and FIL when she was hit on the neck with danda and belt following which her condition start deteriorating. She was taken to a local hospital who referred her to Neurology Department of AIIMS. During her treatment she was put on ventilator and kept in ICU for 2 months.She alleges that even while admitted, she was subjected to multiple episodes of violence by her husband and FIL who threatened to remove the ventilator if her father didn’t pay additional dowry.She was discharged after 6 months with following diagnosis: LMN Quadriparesis with diaphragmatic weakness and ventilator dependence. She was not taken care of properly in her in laws house, so she went back to her own home and currently she is bedridden on ventilator and residing at her parents’ house.She has lodged an FIR with the police alleging domestic violence and torture with demands for additional dowry.A medical board of doctors was formed to opine whether her condition was a result of physical violence meted out her. Her treatment papers were reviewed, and in view of unknown substance administered to the victim, samples were collected for toxicological analysis.Specify the provisions of Indian Penal Code as they apply to the above case.Also mention how the sections are relevant to the case.[/INST]"

# model_input = eval_tokenizer(eval_prompt, return_tensors="pt").to("cuda")

# ft_model.eval()
# with torch.no_grad():
    # generated_output = ft_model.generate(**model_input, max_new_tokens=4096, repetition_penalty=1.15)
    # decoded_output = eval_tokenizer.decode(generated_output[0], skip_special_tokens=True)
    # Remove prompt from decoded output
    # prompt_length = len(eval_prompt)
    # print(decoded_output[prompt_length:].strip())  # Excluding the prompt from the output


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


ction 304B in the indian penal code: ["(1) Where the death of a woman is caused by any burns or bodily injury or occurs otherwise than under normal circumstances within seven years of her marriage and it is shown that soon before her death she was subjected to cruelty or harassment by her husband or any relative of her husband for, or in connection with, any demand for dowry, such death shall be called \\\"dowry death\\\", and such husband or relative shall be deemed to have caused her death.","(2) Whoever commits dowry death shall be punished with imprisonment for a term which shall not be less than seven years but which may extend to imprisonment for life."]


In [ ]:
eval_prompt = "<s>[INST] Heard learned counsel for the applicant and learned A.G.A for the State. By means of this application, the applicant who is involved in case crime no. 213 of 2020, under Sections 498-A/304/326/342/323 IPC, Police Station Kaptanganj, District Kushinagar, is seeking enlargement on bail during the trial. Submission made by learned counsel for the applicant is that F.I.R was registered under sections 498-A/304/326/342/323 IPC against three accused persons Smt. Kaushalya Devi Mother-in-law, Ajay Yadav husband and Rahul Yadav Dewar. Further, before issuing the release order, the sureties be verified. (i) THE APPLICANT SHALL FILE AN UNDERTAKING TO THE EFFECT THAT HE SHALL NOT SEEK ANY ADJOURNMENT ON THE DATE FIXED FOR EVIDENCE WHEN THE WITNESSES ARE PRESENT IN COURT. IN CASE OF DEFAULT OF THIS CONDITION, IT SHALL BE OPEN FOR THE TRIAL COURT TO TREAT IT AS ABUSE OF LIBERTY OF BAIL AND PASS ORDERS IN ACCORDANCE WITH LAW. (ii) THE APPLICANT SHALL REMAIN PRESENT BEFORE THE TRIAL COURT ON EACH DATE FIXED, EITHER PERSONALLY OR THROUGH HIS COUNSEL. IN CASE OF HIS ABSENCE, WITHOUT SUFFICIENT CAUSE, THE TRIAL COURT MAY PROCEED AGAINST HIS UNDER SECTION 229-A IPC. (iii) IN CASE, THE APPLICANT MISUSES THE LIBERTY OF BAIL DURING TRIAL AND IN ORDER TO SECURE HIS PRESENCE PROCLAMATION UNDER SECTION 82 CR.P.C., MAY BE ISSUED AND IF APPLICANT FAILS TO APPEAR BEFORE THE COURT ON THE DATE FIXED IN SUCH PROCLAMATION, THEN, THE TRIAL COURT SHALL INITIATE PROCEEDINGS AGAINST HIM, IN ACCORDANCE WITH LAW, UNDER SECTION 174-A IPC. Since the bail application has been decided under extra-ordinary circumstances, thus in the interest of justice following additional conditions are being imposed just to facilitate the applicant to be released on bail forthwith. Needless to mention that these additional conditions are imposed to cope with emergent condition-: The applicant shall be enlarged on bail on execution of personal bond without sureties till normal functioning of the courts is restored. The computer generated copy of such order shall be self attested by the counsel of the party concerned. The concerned Court/Authority/Official shall verify the authenticity of such computerized copy of the order from the official website of High Court Allahabad and shall make a declaration of such verification in writing. Order Date :- 2.11.2020 Vikram. Specify the provisions of Indian Penal Code as they apply to the above case.Also mention how the sections are relevant to the case.[/INST]"

model_input = eval_tokenizer(eval_prompt, return_tensors="pt").to("cuda")

ft_model.eval()
with torch.no_grad():
    generated_output = ft_model.generate(**model_input, max_new_tokens=4096, repetition_penalty=1.15)
    decoded_output = eval_tokenizer.decode(generated_output[0], skip_special_tokens=True)
    # Remove prompt from decoded output
    prompt_length = len(eval_prompt)
    print(decoded_output[prompt_length:].strip())  # Excluding the prompt from the output

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


ction 498a in the indian penal code: ["Whoever, being the husband or the relative of the husband of a woman, subjects such woman to cruelty shall be punished with imprisonment for a term which may extend to three years and shall also be liable to fine."]
section 304 in the indian penal code: ["If any person commits suicide, whoever abets the commission of such suicide, shall be punished with imprisonment of either description for a term which may extend to ten years, and shall also be liable to fine."]
section 326 in the indian penal code: ["Whoever, except in the case provided for by section 335, voluntarily causes grievous hurt by means of any instrument for shooting, stabbing or cutting, or any instrument which, used as weapon of offence, is likely to cause death, or by means of fire or any heated substance, or by means of any poison or corrosive substance, or by means of any explosive substance, or by means of any substance which it is deleterious to the human body to inhale, to sw

In [ ]:
# eval_prompt = '''<s>[INST]
# This Criminal Original Petition has been filed to quash the criminalproceedings in S.C.No.118 fo 2014 on the file of the learned Chief JudicialMagistrate, Madurai.", "2.The case of the prosecution is that Rani Mangammal Palace siutated atNo.46, Vadakku Chithirai Veedhi was under the control of Public WorksDepartment and the said building had housed the offices of the PWD and therecord room is being maintained on the south-easter corner.", "Behind the saidbuilding, the accused had owned a building and when the first accused wasdemolishing his building on 23.07.2013 at about 4.00 p.m, the accused haddemolished the roof of the said Palace and had caused a loss ofRs.1,50,000/-.", "Therefore, the petitioners were charged for the offences underSection 147, 148, 427, 109 I.P.C and r/w Section 3(1) of TNPPDL Act andSection 30(1)(i)(iv) of the Ancient Monuments and Archaeological Sites andRemains Act, 1958 r/w 149 I.P.C.", "Thereby, they compliedthe said condition.", "Thereafter, the Junior Engineer, Public Works Department(i/c) Building, filed a petition in Crl.M.P.No.192 of 2015 before thelearned Chief Judicial Magistrate, Madurai, in which, he sought permission towithdraw a sum of Rs.2,25,500/- for the purpose of repairing the saidbuilding.", "The learned Chief Judicial Magistrate, Madurai by order dated09.06.2015 in Crl.", "M.P.No.192 of 2015 permitted the Junior Engineer, PublicWorks Department to withdraw the said amount to repair the said Palace.", "Thereafter, the amount has been withdrawn and the said Palace got repaired.", "Therefore, nothing survives to prosecute the petition as alleged by theprosecution.", "4.The learned Government Advocate (Crl.Side) also confirmed the same and stated that the amount has been withdrawn by the Public Works Departments and the said Palace got repaired.", "Further, he has produced the Status Report,dated 11.10.2018 in which, it was stated that the said Palace already gotrepaired.", "5.Heard the learned counsel for the petitioners and the learnedGovernment Advocate (Crl.Side).", "6.Considering the facts and circumstances and also considering thereport submitted by the Public Works Department, the proceedings inS.C.No.118 of 2014 on the file of the Chief Judicial Magistrate Court,Madurai against the petitioners alone shall stand quashed.", "Accordingly, thisCriminal Original Petition is allowed.", "Consequently, connected MiscellaneousPetitions are closed.", "1.The Chief Judicial Magistrate Madurai", "2.The Inspector of Police B1, Vilakkuthoon Police Station Madurai City", "3.The Additional Public Prosecutor Madurai Bench of Madras High Court Madurai.\u001a

# Specify the provisions of Indian Penal Code as they apply to the above case.
# No repetition of points.
# [/INST]'''
# model_input = eval_tokenizer(eval_prompt, return_tensors="pt").to("cuda")

# ft_model.eval()
# with torch.no_grad():
    print(eval_tokenizer.decode(ft_model.generate(**model_input, max_new_tokens=4096, repetition_penalty=1.15)[0], skip_special_tokens=True))


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST]
This Criminal Original Petition has been filed to quash the criminalproceedings in S.C.No.118 fo 2014 on the file of the learned Chief JudicialMagistrate, Madurai.", "2.The case of the prosecution is that Rani Mangammal Palace siutated atNo.46, Vadakku Chithirai Veedhi was under the control of Public WorksDepartment and the said building had housed the offices of the PWD and therecord room is being maintained on the south-easter corner.", "Behind the saidbuilding, the accused had owned a building and when the first accused wasdemolishing his building on 23.07.2013 at about 4.00 p.m, the accused haddemolished the roof of the said Palace and had caused a loss ofRs.1,50,000/-.", "Therefore, the petitioners were charged for the offences underSection 147, 148, 427, 109 I.P.C and r/w Section 3(1) of TNPPDL Act andSection 30(1)(i)(iv) of the Ancient Monuments and Archaeological Sites andRemains Act, 1958 r/w 149 I.P.C.", "Thereby, they compliedthe said condition.", "Thereafter, the Jun